# Procesado de modelos 3D

In [2]:
# Imports
import trimesh
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import pyvista as pv
from PIL import Image
import vtk
import csv
import os

2023-12-27 20:02:58.551929: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-27 20:02:58.734514: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-27 20:02:58.734559: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-27 20:02:58.735498: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-27 20:02:58.823666: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-27 20:02:58.824863: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

Variables globales

In [3]:
# Nombre del archivo CSV
archivo_csv = "matrices_transformacion.csv"

os.remove(archivo_csv)

# Lista de índices de landmarks para nariz, ojos y boca
landmark_indices = [130, 27, 243, 23, 463, 257, 359, 253, 168, 61, 11, 291, 16]

# Crear un objeto FaceLandmarker
base_options = python.BaseOptions(model_asset_path='face_landmarker.task')
VisionRunningMode = mp.tasks.vision.RunningMode
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       running_mode=VisionRunningMode.IMAGE,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

I0000 00:00:1703703783.506084   26990 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1703703783.513099   27071 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.0.4-0ubuntu1~22.04.1), renderer: AMD Radeon Vega 8 Graphics (raven, LLVM 15.0.7, DRM 3.49, 6.2.0-32-generic)
W0000 00:00:1703703783.521943   26990 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### Funciones auxiliares

Función para sacar una imagen 2D dado un objeto en 3D

In [4]:
def take_screenshot(actors, plane, colors = None, textures = None, position = None, window_size = [1024, 768]):
    
    if position is None:
        position = [0.0, 0.0, 0.0]

        if plane.lower() == "xy" or plane.lower() == "yx":
            position[2] = 1000.0
        elif plane.lower() == "yz" or plane.lower() == "zy":
            position[0] = 1000.0
        elif plane.lower() == "xz" or plane.lower() == "zx":
            position[1] = 1000.0 
        else:
            print("ERROR: Wrong plane", plane)
            exit(1)
        

    plotter = pv.Plotter(off_screen = True, window_size = window_size)
    plotter.set_background("white")

    if colors is None:
        colors = [None] * len(actors)

    if textures is None:
        textures = [None] * len(actors)

    for actor, color, tex  in zip(actors, colors, textures):
        _ = plotter.add_mesh(actor, color = color, texture = tex)

    plotter.set_position(position)
    plotter.set_focus([0.0, 0.0, 0.0])
    plotter.enable_parallel_projection()
    plotter.parallel_scale = 200

    return np.array(plotter.screenshot()), plotter

Función para pasar punto 2D a punto 3D

In [5]:
def convert_2d_point_to_3d(pv_mesh, point, plotter):
    
    coordinate = vtk.vtkCoordinate()
    coordinate.SetCoordinateSystemToNormalizedDisplay()

    # the system coordinates is normalized, so 0.5 is the half of the image
    coordinate.SetValue(point[0], point[1])
    world_point = coordinate.GetComputedWorldValue(plotter.renderer)

    start = [world_point[0], -world_point[1], 1000]
    end = [world_point[0], -world_point[1], -1000]

    points, _ = pv_mesh.ray_trace(start, end)

    final_point = None

    if len(points) > 0:
        final_point = points[0]

    return final_point

Código para guardar los puntos en un .pp

In [6]:
def guardar_pp(output_path, coordenadas_3d):
    with open(output_path, "w") as f:
        f.write("<!DOCTYPE PickedPoints>\n")
        f.write("<PickedPoints>\n")
        i = 0
        for idx in coordenadas_3d:
            x, y, z = idx[0], idx[1], idx[2]
            f.write(
                '<point x="{}" y="{}" z="{}" active="1" name="{}" />\n'.format(
                    x, y, z, i
                )
            )
            i += 1
        f.write("</PickedPoints>")

Código para obtener coordenadas de los puntos clave

In [7]:
def obtener_coordenadas(obj_file_path, texture_file_path, output_path):
    # Obtener la imagen del modelo 3D
    pv_mesh = pv.read(obj_file_path)
    texture = pv.read_texture(texture_file_path)
    screenshot, plotter = take_screenshot([pv_mesh], plane='xy', textures=[texture])
    image = Image.fromarray(np.uint8(screenshot))
    image.save('modelo_objetivo.png')

    # Detectamos los face landmarks de la imagen
    imagen = 'modelo_objetivo.png'
    image = mp.Image.create_from_file(imagen)
    detection_result = detector.detect(image)

    puntos = []
    for face_landmarks in detection_result.face_landmarks:
        for idx in landmark_indices:
            landmark = face_landmarks[idx]
            # Crear un vector con coordenadas x, y
            punto_2d = [landmark.x, landmark.y]
            # Agregar el vector a la lista
            puntos.append(punto_2d)
    
    # Calcular la coordenada z
    coordenadas_3d = []

    for point in puntos:
        # Convertir cada punto 2D a 3D utilizando la función proporcionada
        converted_point = convert_2d_point_to_3d(pv_mesh, point, plotter)
        # Agregar el punto convertido a la lista
        coordenadas_3d.append(converted_point)
        
    print("Coordenadas 3D resultantes:", coordenadas_3d)
    guardar_pp(output_path, coordenadas_3d)

    return coordenadas_3d


Función para guardar una matriz en el archivo CSV

In [8]:
def guardar_matriz_en_csv(nombre, matriz, archivo_csv):
    with open(archivo_csv, mode='a', newline='') as file:
        writer = csv.writer(file)
        valores_matriz = [valor for fila in matriz for valor in fila]
        writer.writerow([nombre] + valores_matriz)

### Obtener puntos clave modelo referencia

Cargamos el modelo de referencia 3D

In [9]:
# Ruta del archivo .obj
obj_file_path = 'referencia/referencia.obj'
texture_file_path = 'referencia/referencia.png'
pp_file_path = 'referencia/referencia.pp'

coord_ref = obtener_coordenadas(obj_file_path, texture_file_path, pp_file_path)

Coordenadas 3D resultantes: [array([-50.535995  ,   0.12711287, -17.447363  ], dtype=float32), array([-36.389015 ,  11.032248 ,  -1.3348794], dtype=float32), array([-15.461413 ,   1.2884259,  -4.4633584], dtype=float32), array([-32.29154  ,  -5.1018715,  -8.179023 ], dtype=float32), array([16.348997  ,  0.66007376, -4.3715014 ], dtype=float32), array([37.74045  , 10.050893 , -1.7948587], dtype=float32), array([ 50.77203  ,  -1.9906521, -16.822512 ], dtype=float32), array([32.82299  , -6.5261364, -6.097302 ], dtype=float32), array([ 1.393191 ,  5.5720925, 12.625529 ], dtype=float32), array([-23.518053, -66.90862 ,   2.458813], dtype=float32), array([  2.1766026, -60.613464 ,  17.643604 ], dtype=float32), array([ 25.888952 , -67.48111  ,   2.9253252], dtype=float32), array([  1.8618902, -71.569534 ,  15.830459 ], dtype=float32)]


### Obtener matriz de transformación para otro modelo

In [10]:
archivos = []
# ruta_carpeta = os.getcwd() + "/HeadSpace"
ruta_carpeta = os.getcwd()
for root , _, archs in os.walk(ruta_carpeta):
    for archivo in archs:
        # Verificar si el archivo tiene la extensión .obj
        if archivo.endswith(".obj"):
            ruta_archivo = os.path.join(root, archivo)
            nombre, extension = os.path.splitext(ruta_archivo)
            ruta_relativa = os.path.relpath(nombre, os.getcwd())
            archivos.append(ruta_relativa)
            print(ruta_relativa)

referencia/referencia
HeadSpace/130929144738


In [11]:
for archivo in archivos:
    # Ruta del archivo .obj
    obj_file_path = archivo + ".obj"
    pp_file_path = archivo + ".pp"
    texture_file_path = archivo  + ".bmp"
    
    # Verificar si el archivo .bmp existe
    if not os.path.exists(texture_file_path):
        texture_file_path = archivo + ".png"

    coord_obj = obtener_coordenadas(obj_file_path, texture_file_path, pp_file_path)

    v0 = list(zip(*coord_ref))
    v1 = list(zip(*coord_obj))
    matriz_transformacion = trimesh.transformations.affine_matrix_from_points(v0, v1, shear=False,scale=False)

    for i in range(3):
        matriz_transformacion[i][3] *= -1

    print(matriz_transformacion)

    guardar_matriz_en_csv(obj_file_path, matriz_transformacion, archivo_csv)

Coordenadas 3D resultantes: [array([-50.535995  ,   0.12711287, -17.447363  ], dtype=float32), array([-36.389015 ,  11.032248 ,  -1.3348794], dtype=float32), array([-15.461413 ,   1.2884259,  -4.4633584], dtype=float32), array([-32.29154  ,  -5.1018715,  -8.179023 ], dtype=float32), array([16.348997  ,  0.66007376, -4.3715014 ], dtype=float32), array([37.74045  , 10.050893 , -1.7948587], dtype=float32), array([ 50.77203  ,  -1.9906521, -16.822512 ], dtype=float32), array([32.82299  , -6.5261364, -6.097302 ], dtype=float32), array([ 1.393191 ,  5.5720925, 12.625529 ], dtype=float32), array([-23.518053, -66.90862 ,   2.458813], dtype=float32), array([  2.1766026, -60.613464 ,  17.643604 ], dtype=float32), array([ 25.888952 , -67.48111  ,   2.9253252], dtype=float32), array([  1.8618902, -71.569534 ,  15.830459 ], dtype=float32)]
[[ 1.00000000e+00  1.27420028e-17 -1.28123117e-17 -2.22044605e-16]
 [-2.29407356e-17  1.00000000e+00  6.06101881e-17  3.55271368e-15]
 [ 4.32540806e-18  2.408296